In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import datetime, timedelta
import os
import matplotlib.pyplot as plt

In [22]:
gare11 = yf.Ticker('GARE11.SA')
hist = gare11.history(start='2024-12-16', end='2024-12-18', auto_adjust=True)
print(hist)

                           Open  High   Low  Close  Volume  Dividends  \
Date                                                                    
2024-12-16 00:00:00-03:00  8.23  8.28  8.13   8.13  543796        0.0   
2024-12-17 00:00:00-03:00  8.22  8.22  8.00   8.08  816412        0.0   

                           Stock Splits  
Date                                     
2024-12-16 00:00:00-03:00           0.0  
2024-12-17 00:00:00-03:00           0.0  
